# Capstone Coursera Applied Data Science Week 3

## Part 1 

In [1]:
import pandas as pd
import numpy as np
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [4]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df['Borough'] != 'Not assigned']
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
df2 = df.groupby('Postal Code', sort=False).agg(', '.join)
df2.head(10)

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [6]:
### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
### These two rows will be combined into one row with the neighborhoods separated with a comma
df2.loc[df2['Neighborhood'] =='Not assigned', 'Neighborhood'] = df2.loc[df2['Neighborhood'] =='Not assigned', 'Borough']
df2.reset_index(inplace=True)
df2.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df2.shape


(103, 3)

## Part 2

In [8]:
!pip install geocoder
import geocoder

In [9]:
df2['Latitude'] = None
df2['Longitude'] = None
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


In [10]:
for i, postal_code in enumerate(df2['Postal Code']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    df2.loc[i, 'Latitude'] = latitude
    df2.loc[i, 'Longitude'] = longitude

df2.head(10)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6626,-79.5283
6,M1B,Scarborough,"Malvern, Rouge",43.8114,-79.1966
7,M3B,North York,Don Mills,43.7492,-79.3619
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7072,-79.3119
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378


## Part 3

### Installing geopy & folium

In [11]:
!pip install geopy

In [12]:
!pip install folium
import folium
print('Done')

Done


### Creating Toronto's map

In [13]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='yellow',
        fill=True,
        parse_html=False).add_to(map_toronto)
    
map_toronto

### Importing credentials

In [14]:
CLIENT_ID = 'K23DSBS00GSPJIUSF3VQGCCLLLHIOUSJ0244UPB41GFV4DZL' 
CLIENT_SECRET = 'OJ2S52HI3NQGDZL2IM2JZGX11OBYEG31CKBQZXWV2V3TN4RA'
VERSION = '20201204'
LIMIT = 100

### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Write the code to run the above function on each neighborhood and create a new dataframe

In [16]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Let's check how many venues were returned for each neighborhood

In [17]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,16,16,16,16,16,16
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",19,19,19,19,19,19
Berczy Park,60,60,60,60,60,60
...,...,...,...,...,...,...
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4
Woodbine Heights,18,18,18,18,18,18


### Analysis of each neighborhood

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category¶


In [19]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
2,Bayview Village,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
3,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
4,Berczy Park,0.0,0.0,0.0,0.0,0.016667,0.0,0.016667,0.0,0.000000,...,0.016667,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.016667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
92,Woburn,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.055556,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000
94,York Mills West,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.000000


### Let's put that into a pandas dataframe

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shanghai Restaurant,Supermarket,Bakery,Bubble Tea Shop,Badminton Court,Discount Store,Newsagent,Sushi Restaurant,Department Store
1,"Alderwood, Long Branch",Performing Arts Venue,Convenience Store,Pub,Gym,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room
2,Bayview Village,Construction & Landscaping,Park,Golf Driving Range,Trail,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Sandwich Place,Pharmacy,Juice Bar,Restaurant,Thai Restaurant,Liquor Store,Pub,Indian Restaurant
4,Berczy Park,Coffee Shop,Cheese Shop,Bakery,Farmers Market,Beer Bar,Cocktail Bar,Breakfast Spot,Restaurant,Seafood Restaurant,Pub


### Run k-means to cluster the neighborhood into 5 clusters.


In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.



In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = df2

# merge toronto_grouped with df2 to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7525,-79.3299,2.0,Food & Drink Shop,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm
1,M4A,North York,Victoria Village,43.7306,-79.3131,0.0,Grocery Store,Pharmacy,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Restaurant,Distribution Center,Electronics Store,Event Space,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504,1.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Bookstore,Men's Store,Toy / Game Store,Women's Store,Food Court,American Restaurant,Coffee Shop
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919,1.0,Coffee Shop,Sandwich Place,Café,Gastropub,Theater,Fried Chicken Joint,Italian Restaurant,Bank,Mediterranean Restaurant,Park


### Create Map

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[kclusters-1],
        fill=True,
        fill_color=rainbow[kclusters-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


### Examine cluster

#### Cluster 1

In [25]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0.0,Grocery Store,Pharmacy,Park,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
6,Scarborough,0.0,Zoo Exhibit,Furniture / Home Store,Fast Food Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
12,Scarborough,0.0,Construction & Landscaping,Bar,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Flower Shop
16,York,0.0,Hockey Arena,Field,Trail,Grocery Store,Park,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Donut Shop
17,Etobicoke,0.0,Fish & Chips Shop,College Rec Center,Electronics Store,Shopping Mall,Park,Grocery Store,Farm,Falafel Restaurant,Event Space,Donut Shop
18,Scarborough,0.0,Construction & Landscaping,Park,Gym / Fitness Center,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant
22,Scarborough,0.0,Construction & Landscaping,Coffee Shop,Park,Business Service,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Donut Shop
32,Scarborough,0.0,Park,Spa,Restaurant,Grocery Store,Indian Restaurant,Yoga Studio,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School
35,East York,0.0,Convenience Store,Park,Intersection,Donut Shop,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
39,North York,0.0,Construction & Landscaping,Park,Golf Driving Range,Trail,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant


#### Cluster 2

In [26]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1.0,Coffee Shop,Breakfast Spot,Yoga Studio,Italian Restaurant,Pub,Restaurant,Distribution Center,Electronics Store,Event Space,Food Truck
3,North York,1.0,Clothing Store,Furniture / Home Store,Cosmetics Shop,Bookstore,Men's Store,Toy / Game Store,Women's Store,Food Court,American Restaurant,Coffee Shop
4,Downtown Toronto,1.0,Coffee Shop,Sandwich Place,Café,Gastropub,Theater,Fried Chicken Joint,Italian Restaurant,Bank,Mediterranean Restaurant,Park
5,Etobicoke,1.0,Pharmacy,Park,Grocery Store,Café,Shopping Mall,Skating Rink,Bank,Event Space,Falafel Restaurant,Dumpling Restaurant
7,North York,1.0,Coffee Shop,Intersection,Spa,Bubble Tea Shop,Burger Joint,Smoke Shop,Supermarket,Beer Store,Gas Station,Soccer Field
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1.0,Coffee Shop,Hotel,Café,Restaurant,Gym,Japanese Restaurant,American Restaurant,Seafood Restaurant,Asian Restaurant,Beer Bar
99,Downtown Toronto,1.0,Coffee Shop,Restaurant,Japanese Restaurant,Gay Bar,Sushi Restaurant,Café,Men's Store,Dance Studio,Hotel,Bubble Tea Shop
100,East Toronto,1.0,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Bar,Asian Restaurant,Theater,Sandwich Place,Salon / Barbershop
101,Etobicoke,1.0,Chinese Restaurant,Park,Fast Food Restaurant,Sushi Restaurant,Bank,Italian Restaurant,Coffee Shop,Flower Shop,Deli / Bodega,Electronics Store


#### Cluster 3

In [27]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2.0,Food & Drink Shop,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm
27,North York,2.0,Residential Building (Apartment / Condo),Park,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Yoga Studio
45,North York,2.0,Park,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Donut Shop
49,North York,2.0,Bakery,Park,Yoga Studio,Event Space,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Dumpling Restaurant
68,Central Toronto,2.0,French Restaurant,Park,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm


#### Cluster 4

In [28]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Scarborough,3.0,Trail,Yoga Studio,Event Space,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Donut Shop


#### Cluster 5

In [29]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
71,Scarborough,4.0,Auto Garage,Yoga Studio,Falafel Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Farm,Dumpling Restaurant
